In [ ]:
%%bash

pip install --upgrade pip


In [ ]:
# ig_resnext101_32x8d-224 

In [ ]:
%%bash
time wget https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/seresnext50_32x4d_racm-a304a460.pth
time wget https://github.com/rwightman/pytorch-pretrained-gluonresnet/releases/download/v0.1/gluon_seresnext101_32x4d-cf52900d.pth
time wget https://storage.googleapis.com/tensorflow/keras-applications/nasnet/NASNet-large-no-top.h5
time wget https://storage.googleapis.com/tensorflow/keras-applications/inception_resnet_v2/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5
time wget https://storage.googleapis.com/keras-applications/efficientnetb7_notop.h5
time wget https://storage.googleapis.com/keras-applications/efficientnetb6_notop.h5
time wget https://storage.googleapis.com/keras-applications/efficientnetb5_notop.h5
time wget https://storage.googleapis.com/keras-applications/efficientnetb4_notop.h5
time wget https://storage.googleapis.com/keras-applications/efficientnetb3_notop.h5
time wget https://storage.googleapis.com/keras-applications/efficientnetb2_notop.h5
time wget https://storage.googleapis.com/keras-applications/efficientnetb1_notop.h5
time wget https://storage.googleapis.com/keras-applications/efficientnetb0_notop.h5
time wget https://storage.googleapis.com/tensorflow/keras-applications/mobilenet_v2/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224_no_top.h5
time git clone https://github.com/titu1994/keras-normalized-optimizers.git
time git clone https://github.com/ypeleg/nfnets-keras.git
time git clone https://github.com/sayakpaul/Adaptive-Gradient-Clipping.git
time git clone https://github.com/rwightman/pytorch-image-models.git

ls -hltar
pwd

In [ ]:
!pip install torchvision

In [ ]:
# import torch
# from .resnext_wsl import resnext101_32x48d_wsl

# model=resnext101_32x48d_wsl(progress=True) # example with the ResNeXt-101 32x48d 

# pretrained_dict=torch.load('ResNeXt101_32x48d.pth',map_location='cpu')['model']

# model_dict = model.state_dict()
# for k in model_dict.keys():
#     if(('module.'+k) in pretrained_dict.keys()):
#         model_dict[k]=pretrained_dict.get(('module.'+k))
# model.load_state_dict(model_dict)

In [ ]:
# import timm
# model = timm.create_model(
#     model_name,
#     num_classes=1000,
#     in_chans=3,
#     pretrained=pretrained,
#     checkpoint_path=checkpoint_path)

In [ ]:
if False:
    import requests
    package = requests.get("https://pypi.python.org/pypi/timm/json").json()
    versions = sorted(package["releases"].keys(), reverse=True)
    max_ver = versions[0]
    files = [d['url'] for d in package["releases"][max_ver] if '.whl' in d['url'].lower()]
    files

In [ ]:
!wget https://files.pythonhosted.org/packages/22/c6/ba02d533cec7329323c7d7a317ab49f673846ecef202d4cc40988b6b7786/timm-0.3.4-py3-none-any.whl
# !pip install https://files.pythonhosted.org/packages/22/c6/ba02d533cec7329323c7d7a317ab49f673846ecef202d4cc40988b6b7786/timm-0.3.4-py3-none-any.whl

In [ ]:
!wget https://s3-us-west-2.amazonaws.com/imagenetv2public/imagenetv2-matched-frequency.tar.gz 

In [ ]:
!ls -hltar  .
exit()

In [ ]:
!find imagenetv2-matched-frequency-format-val | wc -l

In [ ]:
import torch
torch.backends.cudnn.benchmark = True

import timm
from timm.data import *
from timm.utils import *

import pandas as pd
import numpy as np
import pynvml
from collections import OrderedDict
import logging
import time

def log_gpu_memory():
    handle = pynvml.nvmlDeviceGetHandleByIndex(0)
    info = pynvml.nvmlDeviceGetMemoryInfo(handle)
    info.free = round(info.free / 1024**2)
    info.used = round(info.used / 1024**2)
    logging.info('GPU memory free: {}, memory used: {}'.format(info.free, info.used))
    return info.used

def get_gpu_memory_total():
    handle = pynvml.nvmlDeviceGetHandleByIndex(0)
    info = pynvml.nvmlDeviceGetMemoryInfo(handle)
    info.total = round(info.total / 1024**2)
    return info.total

setup_default_logging()
    
print('PyTorch version:', torch.__version__)
if torch.cuda.is_available():
    print('CUDA available')
    device='cuda'
else:
    print('CUDA is not available')
    device='cpu'

BATCH_SIZE = 128
if device == 'cuda':
    pynvml.nvmlInit()
    log_gpu_memory()
    total_gpu_mem = get_gpu_memory_total()
    HAS_T4 = False
    if total_gpu_mem > 12300:
        HAS_T4 = True
        logging.info('Running on a T4 GPU or other with > 12GB memory, setting batch size to {}'.format(BATCH_SIZE))
    else:
        BATCH_SIZE = 64
        logging.info('Running on a K80 GPU or other with < 12GB memory, batch size set to {}'.format(BATCH_SIZE))

In [ ]:
ls

In [ ]:
import os
# if not os.path.exists('./imagenetv2-matched-frequency-format-val'):
#     !curl -s https://s3-us-west-2.amazonaws.com/imagenetv2public/imagenetv2-matched-frequency.tar.gz | tar x
# dataset = Dataset('./imagenetv2-matched-frequency-format-val/')
# assert len(dataset) == 10000

In [ ]:
from torchvision.utils import make_grid
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

def show_img(ax, img):
    npimg = img.numpy()
    ax.imshow(np.transpose(npimg, (1,2,0)), interpolation='bicubic')

fig = plt.figure(figsize=(8, 16), dpi=100)
ax = fig.add_subplot('111')
num_images = 4*8
images = []
dataset.transform = transforms.Compose([
    transforms.Resize(320),
    transforms.CenterCrop(320),
    transforms.ToTensor()])
for i in np.random.permutation(np.arange(len(dataset)))[:num_images]:
    images.append(dataset[i][0])

grid_img = make_grid(images, nrow=4, padding=10, normalize=True, scale_each=True)
show_img(ax, grid_img)

In [ ]:
# a basic validation routine and runner that configures each model and loader
from timm.models import TestTimePoolHead

def validate(model, loader, criterion=None, device='cuda'):
    # metrics
    batch_time = timm.utils.AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    
    # for collecting per sample prediction/loss details
    losses_val = []
    top5_idx = []
    top5_val = []
    
    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(loader):
            target = target.to(device)
            input = input.to(device)
            output = model(input)
            
            if criterion is not None:
                loss = criterion(output, target)
                if not loss.size():
                    losses.update(loss.item(), input.size(0))
                else:
                    # only bother collecting top5 we're also collecting per-example loss
                    output = output.softmax(1)
                    top5v, top5i = output.topk(5, 1, True, True)
                    top5_val.append(top5v.cpu().numpy())
                    top5_idx.append(top5i.cpu().numpy())
                    losses_val.append(loss.cpu().numpy())
                    losses.update(loss.mean().item(), input.size(0))
                
            prec1, prec5 = timm.utils.accuracy(output, target, topk=(1, 5))
            top1.update(prec1.item(), input.size(0))
            top5.update(prec5.item(), input.size(0))

            batch_time.update(time.time() - end)
            end = time.time()

            if i % 20 == 0:
                print('Test: [{0}/{1}]\t'
                      'Time {batch_time.val:.3f} ({batch_time.avg:.3f}, {rate_avg:.3f}/s) \t'
                      'Prec@1 {top1.val:.3f} ({top1.avg:.3f})\t'
                      'Prec@5 {top5.val:.3f} ({top5.avg:.3f})'.format(
                    i, len(loader), batch_time=batch_time,
                    rate_avg=input.size(0) / batch_time.avg,
                    top1=top1, top5=top5))

    results = OrderedDict(
        top1=top1.avg, top1_err=100 - top1.avg,
        top5=top5.avg, top5_err=100 - top5.avg,
    )
    if criterion is not None:
        results['loss'] = losses.avg
    if len(top5_idx):
        results['top5_val'] = np.concatenate(top5_val, axis=0)
        results['top5_idx'] = np.concatenate(top5_idx, axis=0)
    if len(losses_val):
        results['losses_val'] = np.concatenate(losses_val, axis=0)
    print(' * Prec@1 {:.3f} ({:.3f}) Prec@5 {:.3f} ({:.3f})'.format(
       results['top1'], results['top1_err'], results['top5'], results['top5_err']))
    return results


def runner(model_args, dataset, device='cuda', collect_loss=False):
    model_name = model_args['model']
    model = timm.create_model(model_name, pretrained=True)
    ttp = False
    if 'ttp' in model_args and model_args['ttp']:
        ttp = True
        logging.info('Applying test time pooling to model')
        model = TestTimePoolHead(model, original_pool=model.default_cfg['pool_size'])
    model = model.to(device)
    model.eval()
    if HAS_T4:
        model = model.half()

    data_config = timm.data.resolve_data_config(model_args, model=model, verbose=True)
        
    loader = timm.data.create_loader(
        dataset,
        input_size=data_config['input_size'],
        batch_size=BATCH_SIZE,
        use_prefetcher=True,
        interpolation='bicubic',
        mean=data_config['mean'],
        std=data_config['std'],
        fp16=HAS_T4,
        crop_pct=1.0 if ttp else data_config['crop_pct'],
        num_workers=2)

    criterion = None
    if collect_loss:
        criterion = torch.nn.CrossEntropyLoss(reduction='none').to(device)
    results = validate(model, loader, criterion, device)
    
    # cleanup checkpoint cache to avoid running out of disk space
    shutil.rmtree(os.path.join(os.environ['HOME'], '.cache', 'torch', 'checkpoints'), True)
    
    # add some non-metric values for charting / comparisons
    results['model'] = model_name
    results['img_size'] = data_config['input_size'][-1]

    # create key to identify model in charts
    key = [model_name, str(data_config['input_size'][-1])]
    if ttp:
        key += ['ttp']
    key = '-'.join(key)
    return key, results

In [ ]:

model_name = 'tf_efficientnet_b7_ns'
model = timm.create_model(model_name, pretrained=True)

In [ ]:
model

In [ ]:
models = [
    dict(model='mobilenetv3_100'),
    dict(model='dpn68b'),
    dict(model='gluon_resnet50_v1d'),
    dict(model='efficientnet_b2'),
    dict(model='gluon_seresnext50_32x4d'),
    dict(model='dpn92'),
    dict(model='gluon_seresnext101_32x4d'),
    dict(model='inception_resnet_v2'),
    dict(model='pnasnet5large'),
    dict(model='tf_efficientnet_b5'),
    dict(model='ig_resnext101_32x8d'),
    dict(model='ig_resnext101_32x16d'),
    dict(model='ig_resnext101_32x32d'),
    dict(model='ig_resnext101_32x48d'),
]

results = OrderedDict()
for ma in models:
    mk, mr = runner(ma, dataset, device)
    results[mk] = mr

results_df = pd.DataFrame.from_dict(results, orient='index')
results_df.to_csv('./cached-results.csv')

In [ ]:
!pip install timm

In [ ]:
t_model = timm.create_model(model_name='tf_efficientnet_b7_ns', pretrained=True, num_classes=1000, in_chans=3, checkpoint_path='', scriptable=None, exportable=None, no_jit=None)

In [ ]:
"""
https://dl.fbaipublicfiles.com/semiweaksupervision/model_files/semi_supervised_resnext101_32x4-dc43570a.pth
""";
import torch
import timm
torch_model = timm.create_model('ssl_resnext101_32x4d', pretrained=True, num_classes=5, global_pool='avg',)
o = model(torch.randn(2, 3, 224, 224))
o.shape

In [ ]:
timm.list_models()

In [ ]:
import torch
import torch.nn as nn
from torchvision.models import vgg16 #https://download.pytorch.org/models/vgg16-397923af.pth
from collections import namedtuple

class Vgg16(torch.nn.Module):
    def __init__(self):
        super(Vgg16, self).__init__()
        features = list(t_model.features)
        print(len(features),'features')
        features = features
        # features的第3，8，15，22层分别是: relu1_2,relu2_2,relu3_3,relu4_3
        self.features = nn.ModuleList(features).eval() 
        
    def forward(self, x):
        results = []
        for ii,model in enumerate(self.features):
            x = model(x)
            if ii in {30,31}:
                results.append(x)
        return results

model = Vgg16()                

In [ ]:
o = t_model.forward(torch.randn(2, 3, 224, 224))
o.shape

In [ ]:
list(t_model.modules())[-2]